# Purpose of this is to extract the data from Radar/Sat datasets and save it as images for pix2pix

### Todo:  make this a script!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
# This is how we can import src module from sibling module
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
print nb_dir
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
from src.wdata import Sat, Radar
from skimage.transform import resize
from sklearn.utils import shuffle
from random import random as random 

C:\Users\Sa\Desktop\capstone-weather


In [2]:
# todo: add code to automatically create directory structure where images will be saved
savedir = os.path.abspath("E:/weatherFall/p2p")
radar_ = [i for mo in [7,8,9] for i in Radar.getMonth(2017,mo)]
bands, radar = [], []
sparsecount = 0
for tup in radar_:
    isSparse = False
    if Radar.isSparse(tup[1], threshold=0.0175): 
        if random() > 0.05:
            continue  # sparse images have a 5% chance of being added
        isSparse = True
    # get corrosponding Satelite image
    ds2 = Sat.getSatFromRad(tup, 2)
    ds4 = Sat.getSatFromRad(tup, 4) 
    ds6 = Sat.getSatFromRad(tup, 6) 
    if ds2 and ds4 and ds6: 
        b2,b4 = Sat.getAndClose(ds2), Sat.getAndClose(ds4)
        b6 = Sat.getAndClose(ds6)
#         bands.append(np.vstack((b2,b4)))
        bands.append(np.stack((b2,b4,b6), axis=2))
        radar.append(resize(tup[1], (52,200)))
        if isSparse: sparsecount += 1

bands, radar = shuffle(bands, radar)

d:\Anaconda2\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
size = len(bands)
print size
print len(bands), len(radar)
print 'NUm of sparse images:', sparsecount

182
182 182
# sparse images: 16


In [4]:
def normBand(arr):
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    return arr
bands = normBand(bands)

In [7]:
def saveImgs():
    for i,d in enumerate(bands[:size]):
        plt.imsave(savedir+"/A/{:>03d}.png".format(i), d, cmap=None)
    print "done bands"
    for i,d in enumerate(radar[:size]):
        plt.imsave(savedir+"/B/{:>03d}.png".format(i), d, cmap='gray', origin='lower')
    print "done radar"
saveImgs()

done bands
done radar
